In [ ]:
import praw
import pandas as pd
from pmaw import PushshiftAPI
import datetime as dt
import requests
import time
import numpy as np

In [ ]:
# Define user agent
user_agent = "praw_scraper_1.0"

# Create an instance of reddit class
reddit = praw.Reddit(username="mawme-grpc",
                     password="redditscrape",
                     client_id="FZLSs0-DoHA2v0kApJNoiQ",
                     client_secret="advGzVds1qSHrvuSEzDFVCGzMvi9lQ",
                     user_agent=user_agent
)


# Create sub-reddit instance
subreddit_name = "deeplearning"
subreddit = reddit.subreddit(subreddit_name)

input_url = "https://www.reddit.com/search/?q=ryanair"

titles=[]
scores=[]
ids=[]
content = []

df = pd.DataFrame()

subreddit = reddit.subreddit('all').search('United Airlines', limit = 1000) # US Airways, American, Southwest, Delta, Virgin America, United Airlines
for submission in subreddit:
    titles.append(submission.title)
    scores.append(submission.upvote_ratio) #upvotes
    ids.append(submission.id)
    content.append(submission.selftext )

df['Title'] = titles
df['Id'] = ids
df['Upvotes'] = scores #upvotes
df['Content'] = content

print(df.shape)
df.head(10)

In [ ]:


api = PushshiftAPI()

df = pd.DataFrame()

errors_days = 0

while df.shape[0] < 9000:
    time.sleep(1)

    start_epoch = None
    end_epoch = None

    if(df.shape[0] == 0):
        start = dt.date.today()
        start = start - dt.timedelta(days = 1 + errors_days)
        start = dt.datetime.fromordinal(start.toordinal())
        start_epoch = int(start.timestamp())

        end = dt.date.today()
        end = dt.datetime.fromordinal(end.toordinal())
        end_epoch = int(end.timestamp())
    else:
        # get last date from df

        
        start = dt.datetime.strptime(df.iloc[-1]['utc_datetime_str'], '%Y-%m-%d %H:%M:%S') # 2023-04-21 15:15:01
        start = start - dt.timedelta(days = 1 + errors_days)
        start = dt.datetime.fromordinal(start.toordinal())
        start_epoch = int(start.timestamp())

        end = dt.datetime.strptime(df.iloc[-1]['utc_datetime_str'], '%Y-%m-%d %H:%M:%S') # 2023-04-21 15:15:01
        end = dt.datetime.fromordinal(end.toordinal())
        end_epoch = int(end.timestamp())

    print('--------------------------- start search at ',start_epoch, ' -----------------------------------------------')
    # Pushsift api directily
    # api_query = 'https://api.pushshift.io/reddit/submission/search/?q=' + 'ryanair' \
    #            + '&subreddit=' + 'flights' \
    #            + '&after=' + str(start_epoch) \
    #            + '&before=' + str(end_epoch) \
    #            + '&size=' + str(900)
    
    # unitedairlines, delta, americanairlines
    api_query = 'https://api.pushshift.io/reddit/submission/search/?' \
                + 'subreddit=' + 'americanairlines' \
                + '&after=' + str(start_epoch) \
                + '&before=' + str(end_epoch) \
                + '&size=' + str(900)

    try:
        r = requests.get(api_query)
        json = r.json()
        df_pushshift = pd.DataFrame(json['data'])
        print('found :', df_pushshift.shape[0], 'total :', df.shape[0], 'title :', df_pushshift.iloc[0]['title'])
        df = df.append(df_pushshift)

        if(df_pushshift.shape[0] == 0):
            errors_days += 1
        else:
            errors_days = 0
    except:
        print('error')
        errors_days += 1

print('---------------------------------------------------------------')
print(f'Pushshift direct df size: {df_pushshift.shape}')
print(df.head())
print('---------------------------------------------------------------')

df.to_csv('americanairlines.csv')  

In [ ]:
df = pd.read_csv('americanairlines.csv')
print(df.shape)
df.head()